In [1]:
import sqlite3
import pandas as pd

### Подключение к БД

In [2]:
conn = sqlite3.connect("data.db")

Выбираем все договоры самой старшей версии с меткой emergency - "Закупка осуществляется вследствие аварии,
 иных чрезвычайных ситуаций природного или техногенного характера, непреодолимой силы,
 принеобходимости срочного медицинского вмешательства, а также для предотвращения угрозы
 возникновения указанных ситуаций"

In [23]:
covid_contracts = pd.read_sql_query(
    """
    SELECT c.guid as contract_guid, c.registration_number as contract_registration_number,
    c.notice_not_placed_by_fz223p5s4 as notice_not_placed_by_fz223p5s4,
    c.extension as contract_extension, c.prolongation as contract_prolongation,
    c.customer_appeale_or_needs_approval as customer_appeale_or_needs_approval,
    c.customer_approval_or_antimonopoly_descision_date as customer_approval_or_antimonopoly_descision_date,
    c.plan_position_guid as contract_plan_position_guid,
    c.create_date_time as contract_create_date_time,
    c.customer_inn as contract_customer_inn, c.placer_inn as contract_placer_inn, c.detached_org_inn as contract_detached_org_inn,
    c.publication_date as contract_publication_date, c.contract_status_code as contract_publication_date,
    c.emergency as emergency, c.provider as provider, c.provider_code as provider_code, c.contract_reg_number as contract_reg_number,
    c.contract_date as contract_date, c.digital_purchase as digital_purchase,
    c.purchase_notice_info_guid as purchase_notice_info_guid, c.purchase_notice_info_number as purchase_notice_info_number,
    c.lot_guid as lot_guid, c.subject_contract as subject_contract, c.purchase_type_code as purchase_type_code,
    c.price as contract_price, c.rub_price as contract_rub_price, c.currency_code as contract_currency_code,
    c.start_execution_date as contract_start_execution_date, c.end_execution_date as contract_end_execution_date,
    c.region_name as region_name
    FROM contract c
    INNER JOIN (
        SELECT contract_reg_number, MAX(version) ver
        FROM contract
        WHERE emergency = 1
        GROUP BY contract_reg_number
    ) tmp ON c.contract_reg_number = tmp.contract_reg_number AND c.version = tmp.ver
    """,
    conn
)

covid_contracts

,contract_guid,contract_registration_number,notice_not_placed_by_fz223p5s4,contract_extension,contract_prolongation,customer_appeale_or_needs_approval,customer_approval_or_antimonopoly_descision_date,contract_plan_position_guid,contract_create_date_time,contract_customer_inn,...,purchase_notice_info_number,lot_guid,subject_contract,purchase_type_code,contract_price,contract_rub_price,contract_currency_code,contract_start_execution_date,contract_end_execution_date,region_name
0,00015975-9c32-4b00-922d-7ab2866d3cf7,51433000147200015550001,1,0,0,0,None,None,2020-12-01 00:32:23,1433000147,...,None,None,None,30000,5.342400e+06,NaN,RUB,2020-10-15,2020-12-17,Saha_Jakutiya_Resp
1,0008f92e-3856-4383-97d2-2fd8b9b71aee,57708503727200082220007,1,0,0,0,None,None,2021-09-23 12:34:16,7708503727,...,None,None,Выполнение комплекса аварийно-восстановительны...,30000,1.016760e+09,1.016760e+09,RUB,2020-05-31,2021-12-30,Moskva
2,000dc3f6-e39c-456b-b321-5486e2d8b78e,76652022897210001260001,1,0,0,0,None,40451834-df9b-11eb-b80c-b8cef612661b,2021-07-23 04:46:32,6652022897,...,None,None,1719-2021-00259. Выполнение работ по ремонту э...,30000,1.194000e+05,1.194000e+05,RUB,2021-07-21,2021-08-20,Sverdlovskaya_obl
3,000efbce-29cc-4fda-ba27-e5b895f5b6ad,56684021776200003580001,0,1,0,0,None,None,2020-12-02 05:02:53,6684021776,...,32009740825,0621d71c-65f9-4181-81d5-b9e077fab7ef,Поставка лекарственных препаратов,40000,2.453075e+06,2.453075e+06,RUB,2020-11-29,2021-02-27,Sverdlovskaya_obl
4,000f310c-5934-4d63-be2e-74dd4c99e301,56611000527200002950001,1,0,0,0,None,ed850d8e-b314-4c08-92b1-e49429220e75,2020-10-27 06:07:26,6611000527,...,None,None,Поставка СИЗ: перчатки,30000,2.408000e+05,2.408000e+05,RUB,2020-10-25,2020-12-30,Sverdlovskaya_obl
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16638,fff92ffe-8c3f-443f-8ca1-54b8cb98bb64,53801013663210002050001,1,0,0,0,None,None,2021-10-04 02:31:36,3801013663,...,None,None,Поставка лекарственного препарата (гепарин),30000,1.341640e+05,NaN,RUB,2021-10-03,2021-10-30,Irkutskaya_obl
16639,fffa65c3-f306-4bf8-a55c-89d740483fc5,57702070139200015180001,1,0,0,0,None,None,2020-06-04 06:38:35,7702070139,...,None,None,Поставка масок одноразовых,30000,2.800000e+06,NaN,RUB,2020-05-18,2020-06-03,Sankt-Peterburg
16640,fffaff02-11f4-4936-9d2c-01cc2b4c1c8d,58601053210200000270001,1,0,0,0,None,None,2020-10-02 05:04:25,8601053210,...,None,None,неисключительное право на воспроизведение прог...,30000,1.250372e+05,NaN,RUB,2020-10-01,2020-12-30,Hanty-Mansiiskii_AO_Iugra_AO
16641,fffb9623-244a-4c27-b1c7-4518ab6412bd,51431004812210000980001,0,0,0,0,None,None,2021-07-07 09:05:27,1431004812,...,32110451253,89d200ae-e338-463b-89b3-0cf5955eee3c,Поставка лекарственных средств (артлегиа),40000,6.006000e+05,NaN,RUB,2021-07-06,2021-12-30,Saha_Jakutiya_Resp


Для дальнейшего анализа необходимо проверить наличие поля с указанием лота

In [24]:
covid_contracts_with_not_null_lot = covid_contracts[covid_contracts["lot_guid"].notnull()]

print(f'Количество "ковидных" договоров: {covid_contracts.shape[0]}')
print(f'Количество "ковидных" договоров с указанием лота: {covid_contracts_with_not_null_lot.shape[0]}')

Количество "ковидных" договоров: 16643
Количество "ковидных" договоров с указанием лота: 7699


Договоры без указания lot_guid, это те где извещение не размещалось в соответствии
с частью 5 статьи 4 Федерального Закона № 223-ФЗ, например https://zakupki.gov.ru/epz/contractfz223/card/contract-info.html?id=9268727

Для выборки будем использовать только те договоры, в которых присутствуют лоты

Проверим, существуют ли лоты, на которые было заключено несколько договоров.

In [5]:
lots_with_many_contracts = pd.read_sql_query(
    """
    SELECT * FROM (
        SELECT lot_guid, COUNT(*) as c
            FROM contract c
            INNER JOIN (
                SELECT contract_reg_number, MAX(version) ver
                FROM contract
                WHERE emergency = 1
                and lot_guid is not null
                GROUP BY contract_reg_number
            ) tmp ON c.contract_reg_number = tmp.contract_reg_number AND c.version = tmp.ver
        GROUP BY lot_guid
    ) WHERE c > 1
    """,
    conn
)

lots_with_many_contracts

,lot_guid,c
0,08fcfa39-dd35-45fc-a7fe-8856ab5a7f68,2
1,0a0bad0e-049a-43e9-adfc-6896a21b42ed,3
2,1097ee47-fe1d-4bce-8736-51e24e7c9910,2
3,11db9fb9-da1b-49fc-a31e-97462414707f,35
4,1581b86c-4c98-40d3-b578-76c76f4a2670,2
5,161fae5d-952d-4fee-9bbd-3ff29b00fa30,4
6,1c475aea-1c94-4fb8-913f-54d61fc719ed,25
7,25ba1976-d000-4ee0-922c-ae463b64668b,2
8,2b2b33b4-a466-4bf1-8632-436a0998ed67,19
9,3032623b-bcf8-4b91-b93d-03010e4a41c3,2


Такие договоры существуют.  Необходимо будет добавить информацию об этом в выборку.

Выберем из базы договоры с лотами

In [6]:
covid_contracts_with_lot = pd.read_sql_query(
    """
    SELECT c.guid as contract_guid, c.registration_number as contract_registration_number,
    c.notice_not_placed_by_fz223p5s4 as notice_not_placed_by_fz223p5s4,
    c.extension as contract_extension, c.prolongation as contract_prolongation,
    c.customer_appeale_or_needs_approval as customer_appeale_or_needs_approval,
    c.customer_approval_or_antimonopoly_descision_date as customer_approval_or_antimonopoly_descision_date,
    c.plan_position_guid as contract_plan_position_guid,
    c.create_date_time as contract_create_date_time,
    c.customer_inn as contract_customer_inn, c.placer_inn as contract_placer_inn, c.detached_org_inn as contract_detached_org_inn,
    c.publication_date as contract_publication_date, c.contract_status_code as contract_publication_date,
    c.emergency as emergency, c.provider as provider, c.provider_code as provider_code, c.contract_reg_number as contract_reg_number,
    c.contract_date as contract_date, c.digital_purchase as digital_purchase,
    c.purchase_notice_info_guid as purchase_notice_info_guid, c.purchase_notice_info_number as purchase_notice_info_number,
    c.lot_guid as lot_guid, c.subject_contract as subject_contract, c.purchase_type_code as purchase_type_code,
    c.price as contract_price, c.rub_price as contract_rub_price, c.currency_code as contract_currency_code,
    c.start_execution_date as contract_start_execution_date, c.end_execution_date as contract_end_execution_date,
    c.region_name as region_name,
    pnld.initial_sum as notice_lot_initial_sum, pnld.starting_contract_price_rub as notice_lot_starting_contract_price_rub,
    pnld.currency_code as notice_lot_currency_code,
    pnld.commodity_price as notice_lot_commodity_price, pnld.commodity_price_rub as notice_lot_commodity_price_rub,
    pnld.max_contract_price as notice_lot_max_contract_price, pnld.max_contract_price_rub as notice_lot_max_contract_price_rub
    FROM contract c
    INNER JOIN (
        SELECT contract_reg_number, MAX(version) ver
        FROM contract
        WHERE emergency = 1
        AND lot_guid is not null
        GROUP BY contract_reg_number
    ) tmp ON c.contract_reg_number = tmp.contract_reg_number AND c.version = tmp.ver
    INNER JOIN purchase_notice_lot pnl ON c.lot_guid = pnl.guid
    LEFT JOIN purchase_notice_lot_data pnld on pnl.guid = pnld.purchase_notice_lot_guid
    """,
    conn
)

covid_contracts_with_lot

,contract_guid,contract_registration_number,notice_not_placed_by_fz223p5s4,contract_extension,contract_prolongation,customer_appeale_or_needs_approval,customer_approval_or_antimonopoly_descision_date,contract_plan_position_guid,contract_create_date_time,contract_customer_inn,...,contract_start_execution_date,contract_end_execution_date,region_name,notice_lot_initial_sum,notice_lot_starting_contract_price_rub,notice_lot_currency_code,notice_lot_commodity_price,notice_lot_commodity_price_rub,notice_lot_max_contract_price,notice_lot_max_contract_price_rub
0,000efbce-29cc-4fda-ba27-e5b895f5b6ad,56684021776200003580001,0,1,0,0,None,None,2020-12-02 05:02:53,6684021776,...,2020-11-29,2021-02-27,Sverdlovskaya_obl,2453075.00,NaN,RUB,None,None,NaN,None
1,0010055e-61aa-42c1-9ae5-cfadf002275c,76660000400200006120001,0,0,0,0,None,None,2020-08-12 04:13:04,6660000400,...,2020-08-11,2020-12-30,Sverdlovskaya_obl,6000000.00,NaN,RUB,None,None,NaN,None
2,001170e0-57c5-4d89-89f2-e607079cd321,53821005886200000390001,0,0,0,0,None,None,2020-08-13 00:50:19,3821005886,...,2020-08-11,2020-12-30,Irkutskaya_obl,243570.00,NaN,RUB,None,None,NaN,None
3,00161a73-4f64-432b-915a-6d751c91d5a1,58603076205200000240001,0,0,0,0,None,None,2020-12-19 17:15:48,8603076205,...,2020-12-16,2020-12-30,Hanty-Mansiiskii_AO_Iugra_AO,85820.00,NaN,RUB,None,None,NaN,None
4,0019ca0a-9df7-43fe-a57a-1a76ace715c4,83525147496200001260001,0,0,0,0,None,None,2020-11-16 08:51:47,3525147496,...,2020-11-12,None,Vologodskaya_obl,358403.76,NaN,RUB,None,None,NaN,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7641,ffd13b8b-ffef-4b0d-8fa5-3214f3e44e2b,52312210286200000990001,0,0,0,0,None,None,2020-12-08 11:26:50,2312210286,...,2020-07-29,None,Krasnodarskii_krai,294928.00,NaN,RUB,None,None,NaN,None
7642,ffda639d-aada-44de-974a-4ce9ac345a55,67723011906200009850003,0,0,0,0,None,None,2020-11-11 10:04:03,7723011906,...,2020-10-25,2020-12-30,Moskva,557776.76,NaN,RUB,None,None,NaN,None
7643,ffe52c52-50b9-4962-8392-28ab996f7d62,56829018032200000820001,0,0,0,0,None,None,2020-07-15 06:35:10,6829018032,...,2020-07-13,2020-08-30,Tambovskaya_obl,34650000.00,NaN,RUB,None,None,NaN,None
7644,ffeed1b5-bd37-4706-8109-95a9367e499e,56731033838200000460001,0,0,0,0,None,None,2020-12-21 07:26:42,6731033838,...,2020-12-17,2020-12-27,Smolenskaya_obl,895400.00,NaN,RUB,None,None,NaN,None


Проверим, существуют ли договоры, у которых заказчик отличается от заказчика в извещении

In [7]:
contracts_with_different_customer = pd.read_sql_query(
    """
    SELECT c.guid, c.customer_inn, c.placer_inn, c.detached_org_inn, pn.guid, pn.customer_inn, pn.placer_inn, pn.detached_org_inn
    FROM contract c
    INNER JOIN purchase_notice_lot pnl on c.lot_guid = pnl.guid
    LEFT JOIN purchase_notice_to_lot pntl on pnl.guid = pntl.purchase_notice_lot_guid
    LEFT JOIN purchase_notice pn on pntl.purchase_notice_guid = pn.guid
    WHERE
    c.lot_guid is not null
    AND c.emergency = 1
    AND (c.customer_inn != pn.customer_inn or c.placer_inn != pn.placer_inn)
    ;
    """,
    conn
)

contracts_with_different_customer

,guid,customer_inn,placer_inn,detached_org_inn,guid,customer_inn,placer_inn,detached_org_inn
0,02857c3c-59a6-44be-b942-4364a4c11a89,1435149216,1435149216,1435149216,de0960c6-fc19-403b-b852-7bca159cc3e5,1435149216,7710357167,None
1,02a74aef-0dc0-48fb-b0d3-70731900673d,7736182930,7736182930,7736182930,84925ecd-c713-420e-b1a4-683af576b9e9,7736182930,7707308480,None
2,06c265e1-678b-4668-a263-54c8a9bcf63b,6345012488,6345012488,6345012488,1d063c20-7aa0-46ad-9289-a1b9bfb28b87,1510012774,1510012774,None
3,06c265e1-678b-4668-a263-54c8a9bcf63b,6345012488,6345012488,6345012488,219cfb41-11cb-aae8-b709-0d5a6bd131db,1510012774,1510012774,None
4,0c62cca5-11ff-4c74-98b3-99bfba43f439,7736182930,7736182930,7736182930,86da896b-2ac2-40d9-b8fd-5afc5c6834ef,7736182930,7707308480,None
...,...,...,...,...,...,...,...,...
151,fb713851-be44-4799-9bd0-902ae17156aa,1435149216,1435149216,1435149216,0c28ef37-7a49-41ae-aa0a-9c5fcf3c7dcc,1435149216,7710357167,None
152,fe2b0f45-b439-47ba-98ec-b603db403de6,1101460616,1101460616,1101460616,114a6b3b-0786-4e3a-b781-0ea6bf86620f,1101460616,7707308480,None
153,fe2b0f45-b439-47ba-98ec-b603db403de6,1101460616,1101460616,1101460616,4902cf8b-8311-41e0-9ecf-f05640097c81,1101460616,7707308480,None
154,fe33caf4-1d35-4e0a-80ba-9706164e3018,2308131994,2308131994,2308131994,ff0817d0-30ce-4169-921e-75834f01b2d2,2308131994,7707308480,None


Такие договоры существуют, поэтому необходимо будет добавить данные об изначальном заказчике в выборку

Дополним информацией о том, был ли договор завершен или расторгнут

In [8]:
covid_contracts_performance_with_lot = pd.read_sql_query(
    """
    SELECT c.guid as contract_guid, c.registration_number as contract_registration_number,
    c.notice_not_placed_by_fz223p5s4 as notice_not_placed_by_fz223p5s4,
    c.extension as contract_extension, c.prolongation as contract_prolongation,
    c.customer_appeale_or_needs_approval as customer_appeale_or_needs_approval,
    c.customer_approval_or_antimonopoly_descision_date as customer_approval_or_antimonopoly_descision_date,
    c.plan_position_guid as contract_plan_position_guid,
    c.create_date_time as contract_create_date_time,
    c.customer_inn as contract_customer_inn, c.placer_inn as contract_placer_inn, c.detached_org_inn as contract_detached_org_inn,
    c.publication_date as contract_publication_date, c.contract_status_code as contract_publication_date,
    c.emergency as emergency, c.provider as provider, c.provider_code as provider_code, c.contract_reg_number as contract_reg_number,
    c.contract_date as contract_date, c.digital_purchase as digital_purchase,
    c.purchase_notice_info_guid as purchase_notice_info_guid, c.purchase_notice_info_number as purchase_notice_info_number,
    c.lot_guid as lot_guid, c.subject_contract as subject_contract, c.purchase_type_code as purchase_type_code,
    c.price as contract_price, c.rub_price as contract_rub_price, c.currency_code as contract_currency_code,
    c.start_execution_date as contract_start_execution_date, c.end_execution_date as contract_end_execution_date,
    c.region_name as region_name,
    pnld.initial_sum as notice_lot_initial_sum, pnld.starting_contract_price_rub as notice_lot_starting_contract_price_rub,
    pnld.currency_code as notice_lot_currency_code,
    pnld.commodity_price as notice_lot_commodity_price, pnld.commodity_price_rub as notice_lot_commodity_price_rub,
    pnld.max_contract_price as notice_lot_max_contract_price, pnld.max_contract_price_rub as notice_lot_max_contract_price_rub,
    cp.is_complete as contract_is_complete
    FROM contract c
    INNER JOIN (
        SELECT contract_reg_number, MAX(version) ver
        FROM contract
        WHERE emergency = 1
        AND lot_guid is not null
        GROUP BY contract_reg_number
    ) tmp ON c.contract_reg_number = tmp.contract_reg_number AND c.version = tmp.ver
    INNER JOIN purchase_notice_lot pnl ON c.lot_guid = pnl.guid
    LEFT JOIN purchase_notice_lot_data pnld on pnl.guid = pnld.purchase_notice_lot_guid
    LEFT JOIN (
        SELECT c2.contract_reg_number, MIN(is_complete) as is_complete
        FROM contract c2
        LEFT JOIN contract_performance cp2 ON c2.contract_reg_number = cp2.contract_registration_number
        WHERE c2.emergency = 1
        AND c2.lot_guid is not null
        GROUP BY c2.contract_reg_number
    ) cp ON cp.contract_reg_number = c.contract_reg_number
    """,
    conn
)

covid_contracts_performance_with_lot

,contract_guid,contract_registration_number,notice_not_placed_by_fz223p5s4,contract_extension,contract_prolongation,customer_appeale_or_needs_approval,customer_approval_or_antimonopoly_descision_date,contract_plan_position_guid,contract_create_date_time,contract_customer_inn,...,contract_end_execution_date,region_name,notice_lot_initial_sum,notice_lot_starting_contract_price_rub,notice_lot_currency_code,notice_lot_commodity_price,notice_lot_commodity_price_rub,notice_lot_max_contract_price,notice_lot_max_contract_price_rub,contract_is_complete
0,000efbce-29cc-4fda-ba27-e5b895f5b6ad,56684021776200003580001,0,1,0,0,None,None,2020-12-02 05:02:53,6684021776,...,2021-02-27,Sverdlovskaya_obl,2453075.00,NaN,RUB,None,None,NaN,None,1.0
1,0010055e-61aa-42c1-9ae5-cfadf002275c,76660000400200006120001,0,0,0,0,None,None,2020-08-12 04:13:04,6660000400,...,2020-12-30,Sverdlovskaya_obl,6000000.00,NaN,RUB,None,None,NaN,None,1.0
2,001170e0-57c5-4d89-89f2-e607079cd321,53821005886200000390001,0,0,0,0,None,None,2020-08-13 00:50:19,3821005886,...,2020-12-30,Irkutskaya_obl,243570.00,NaN,RUB,None,None,NaN,None,1.0
3,00161a73-4f64-432b-915a-6d751c91d5a1,58603076205200000240001,0,0,0,0,None,None,2020-12-19 17:15:48,8603076205,...,2020-12-30,Hanty-Mansiiskii_AO_Iugra_AO,85820.00,NaN,RUB,None,None,NaN,None,1.0
4,0019ca0a-9df7-43fe-a57a-1a76ace715c4,83525147496200001260001,0,0,0,0,None,None,2020-11-16 08:51:47,3525147496,...,None,Vologodskaya_obl,358403.76,NaN,RUB,None,None,NaN,None,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7641,ffd13b8b-ffef-4b0d-8fa5-3214f3e44e2b,52312210286200000990001,0,0,0,0,None,None,2020-12-08 11:26:50,2312210286,...,None,Krasnodarskii_krai,294928.00,NaN,RUB,None,None,NaN,None,NaN
7642,ffda639d-aada-44de-974a-4ce9ac345a55,67723011906200009850003,0,0,0,0,None,None,2020-11-11 10:04:03,7723011906,...,2020-12-30,Moskva,557776.76,NaN,RUB,None,None,NaN,None,1.0
7643,ffe52c52-50b9-4962-8392-28ab996f7d62,56829018032200000820001,0,0,0,0,None,None,2020-07-15 06:35:10,6829018032,...,2020-08-30,Tambovskaya_obl,34650000.00,NaN,RUB,None,None,NaN,None,1.0
7644,ffeed1b5-bd37-4706-8109-95a9367e499e,56731033838200000460001,0,0,0,0,None,None,2020-12-21 07:26:42,6731033838,...,2020-12-27,Smolenskaya_obl,895400.00,NaN,RUB,None,None,NaN,None,NaN


Необходимо дополнить выборку информацией о позициях договора

In [9]:
covid_contracts_positions_with_lot = pd.read_sql_query(
    """
    SELECT c.guid as contract_guid, c.registration_number as contract_registration_number,
    c.notice_not_placed_by_fz223p5s4 as notice_not_placed_by_fz223p5s4,
    c.extension as contract_extension, c.prolongation as contract_prolongation,
    c.customer_appeale_or_needs_approval as customer_appeale_or_needs_approval,
    c.customer_approval_or_antimonopoly_descision_date as customer_approval_or_antimonopoly_descision_date,
    c.plan_position_guid as contract_plan_position_guid,
    c.create_date_time as contract_create_date_time,
    c.customer_inn as contract_customer_inn, c.placer_inn as contract_placer_inn, c.detached_org_inn as contract_detached_org_inn,
    c.publication_date as contract_publication_date, c.contract_status_code as contract_publication_date,
    c.emergency as emergency, c.provider as provider, c.provider_code as provider_code, c.contract_reg_number as contract_reg_number,
    c.contract_date as contract_date, c.digital_purchase as digital_purchase,
    c.purchase_notice_info_guid as purchase_notice_info_guid, c.purchase_notice_info_number as purchase_notice_info_number,
    c.lot_guid as lot_guid, c.subject_contract as subject_contract, c.purchase_type_code as purchase_type_code,
    c.price as contract_price, c.rub_price as contract_rub_price, c.currency_code as contract_currency_code,
    c.start_execution_date as contract_start_execution_date, c.end_execution_date as contract_end_execution_date,
    c.region_name as region_name,
    pnld.initial_sum as notice_lot_initial_sum, pnld.starting_contract_price_rub as notice_lot_starting_contract_price_rub,
    pnld.currency_code as notice_lot_currency_code,
    pnld.commodity_price as notice_lot_commodity_price, pnld.commodity_price_rub as notice_lot_commodity_price_rub,
    pnld.max_contract_price as notice_lot_max_contract_price, pnld.max_contract_price_rub as notice_lot_max_contract_price_rub,
    cp.is_complete as contract_is_complete,
    cpos.name as contract_position_name,
    cpos.okpd2_code as position_okpd2_code, cpos.okpd2_name as position_okpd2_name,
    cpos.country_code as position_country_code, cpos.producer_country as position_producer_country,
    cpos.impossible_to_determine_attr as position_impossible_to_determine_attr, cpos.okei_code as position_okei_code,
    cpos.okei_name as position_okei_name, cpos.qty as position_qty, cpos.unit_price as position_unit_price,
    cpos.rub_unit_price as position_rub_unit_price, cpos.currency_code as position_currency_code
    FROM contract c
    INNER JOIN (
        SELECT contract_reg_number, MAX(version) ver
        FROM contract
        WHERE emergency = 1
        AND lot_guid is not null
        GROUP BY contract_reg_number
    ) tmp ON c.contract_reg_number = tmp.contract_reg_number AND c.version = tmp.ver
    INNER JOIN purchase_notice_lot pnl ON c.lot_guid = pnl.guid
    LEFT JOIN purchase_notice_lot_data pnld on pnl.guid = pnld.purchase_notice_lot_guid
    LEFT JOIN (
        SELECT c2.contract_reg_number, MIN(is_complete) as is_complete
        FROM contract c2
        LEFT JOIN contract_performance cp2 ON c2.contract_reg_number = cp2.contract_registration_number
        WHERE c2.emergency = 1
        AND c2.lot_guid is not null
        GROUP BY c2.contract_reg_number
    ) cp ON cp.contract_reg_number = c.contract_reg_number
    LEFT JOIN position_to_contract ptc on c.guid = ptc.contract_guid
    LEFT JOIN contract_position cpos on ptc.position_guid = cpos.guid
    """,
    conn
)

covid_contracts_positions_with_lot

,contract_guid,contract_registration_number,notice_not_placed_by_fz223p5s4,contract_extension,contract_prolongation,customer_appeale_or_needs_approval,customer_approval_or_antimonopoly_descision_date,contract_plan_position_guid,contract_create_date_time,contract_customer_inn,...,position_okpd2_name,position_country_code,position_producer_country,position_impossible_to_determine_attr,position_okei_code,position_okei_name,position_qty,position_unit_price,position_rub_unit_price,position_currency_code
0,000efbce-29cc-4fda-ba27-e5b895f5b6ad,56684021776200003580001,0,1,0,0,None,None,2020-12-02 05:02:53,6684021776,...,Препараты антибактериальные для системного исп...,None,None,0,778,Упаковка,700.0,750.0,NaN,None
1,000efbce-29cc-4fda-ba27-e5b895f5b6ad,56684021776200003580001,0,1,0,0,None,None,2020-12-02 05:02:53,6684021776,...,Препараты антибактериальные для системного исп...,None,None,0,778,Упаковка,200.0,1600.0,NaN,None
2,000efbce-29cc-4fda-ba27-e5b895f5b6ad,56684021776200003580001,0,1,0,0,None,None,2020-12-02 05:02:53,6684021776,...,Препараты антибактериальные для системного исп...,None,None,0,778,Упаковка,500.0,940.0,NaN,None
3,000efbce-29cc-4fda-ba27-e5b895f5b6ad,56684021776200003580001,0,1,0,0,None,None,2020-12-02 05:02:53,6684021776,...,Антикоагулянты,None,None,0,778,Упаковка,66.0,2595.0,NaN,None
4,000efbce-29cc-4fda-ba27-e5b895f5b6ad,56684021776200003580001,0,1,0,0,None,None,2020-12-02 05:02:53,6684021776,...,Препараты антибактериальные для системного исп...,None,None,0,778,Упаковка,6.0,17050.0,NaN,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11305,ffd13b8b-ffef-4b0d-8fa5-3214f3e44e2b,52312210286200000990001,0,0,0,0,None,None,2020-12-08 11:26:50,2312210286,...,Работы строительные по прокладке местных трубо...,None,None,1,None,None,NaN,NaN,NaN,None
11306,ffda639d-aada-44de-974a-4ce9ac345a55,67723011906200009850003,0,0,0,0,None,None,2020-11-11 10:04:03,7723011906,...,Услуги организаций санитарно-эпидемиологическо...,None,None,0,796,Штука,44.0,NaN,NaN,None
11307,ffe52c52-50b9-4962-8392-28ab996f7d62,56829018032200000820001,0,0,0,0,None,None,2020-07-15 06:35:10,6829018032,...,Одежда производственная и профессиональная про...,None,None,1,None,None,NaN,NaN,NaN,None
11308,ffeed1b5-bd37-4706-8109-95a9367e499e,56731033838200000460001,0,0,0,0,None,None,2020-12-21 07:26:42,6731033838,...,Комбинезоны мужские производственные и професс...,None,None,0,796,Штука,814.0,NaN,NaN,None


Добавим информацию о том, сколько договоров заключено по лоту в выборку.

In [10]:
contracts_count_for_lot = pd.read_sql_query(
    """
    SELECT lot_guid, COUNT(*) as contracts_for_lot
        FROM contract c
        INNER JOIN (
            SELECT contract_reg_number, MAX(version) ver
            FROM contract
            WHERE emergency = 1
            AND lot_guid is not null
            GROUP BY contract_reg_number
        ) tmp ON c.contract_reg_number = tmp.contract_reg_number AND c.version = tmp.ver
    GROUP BY lot_guid
    """,
    conn
)

contracts_count_for_lot

,lot_guid,contracts_for_lot
0,00069375-47f6-40d8-8b20-df78140b1416,1
1,001180aa-ec39-fe24-e63b-de59c6c8b320,1
2,0017ef8e-08d7-4175-a7fd-059f347b92a2,1
3,001b3f81-61cb-44be-85c6-c54ad269bc50,1
4,0024ab3a-2d0b-1777-ac85-fe8b9ab39c2d,1
...,...,...
7483,ffe5b98d-3546-41ac-9caf-ec25916054ca,1
7484,ffea1521-4052-4501-ab6f-46b57d785c44,1
7485,fff0fffc-00bc-49c4-a131-fcc41d1f70dd,1
7486,fff3b4ba-20f9-451e-8ecd-d0c185e2cf56,1


In [11]:
dataset = pd.merge(
    covid_contracts_positions_with_lot,
    contracts_count_for_lot,
    how='left',
    on=['lot_guid']
)

dataset

,contract_guid,contract_registration_number,notice_not_placed_by_fz223p5s4,contract_extension,contract_prolongation,customer_appeale_or_needs_approval,customer_approval_or_antimonopoly_descision_date,contract_plan_position_guid,contract_create_date_time,contract_customer_inn,...,position_country_code,position_producer_country,position_impossible_to_determine_attr,position_okei_code,position_okei_name,position_qty,position_unit_price,position_rub_unit_price,position_currency_code,contracts_for_lot
0,000efbce-29cc-4fda-ba27-e5b895f5b6ad,56684021776200003580001,0,1,0,0,None,None,2020-12-02 05:02:53,6684021776,...,None,None,0,778,Упаковка,700.0,750.0,NaN,None,1
1,000efbce-29cc-4fda-ba27-e5b895f5b6ad,56684021776200003580001,0,1,0,0,None,None,2020-12-02 05:02:53,6684021776,...,None,None,0,778,Упаковка,200.0,1600.0,NaN,None,1
2,000efbce-29cc-4fda-ba27-e5b895f5b6ad,56684021776200003580001,0,1,0,0,None,None,2020-12-02 05:02:53,6684021776,...,None,None,0,778,Упаковка,500.0,940.0,NaN,None,1
3,000efbce-29cc-4fda-ba27-e5b895f5b6ad,56684021776200003580001,0,1,0,0,None,None,2020-12-02 05:02:53,6684021776,...,None,None,0,778,Упаковка,66.0,2595.0,NaN,None,1
4,000efbce-29cc-4fda-ba27-e5b895f5b6ad,56684021776200003580001,0,1,0,0,None,None,2020-12-02 05:02:53,6684021776,...,None,None,0,778,Упаковка,6.0,17050.0,NaN,None,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11305,ffd13b8b-ffef-4b0d-8fa5-3214f3e44e2b,52312210286200000990001,0,0,0,0,None,None,2020-12-08 11:26:50,2312210286,...,None,None,1,None,None,NaN,NaN,NaN,None,1
11306,ffda639d-aada-44de-974a-4ce9ac345a55,67723011906200009850003,0,0,0,0,None,None,2020-11-11 10:04:03,7723011906,...,None,None,0,796,Штука,44.0,NaN,NaN,None,1
11307,ffe52c52-50b9-4962-8392-28ab996f7d62,56829018032200000820001,0,0,0,0,None,None,2020-07-15 06:35:10,6829018032,...,None,None,1,None,None,NaN,NaN,NaN,None,1
11308,ffeed1b5-bd37-4706-8109-95a9367e499e,56731033838200000460001,0,0,0,0,None,None,2020-12-21 07:26:42,6731033838,...,None,None,0,796,Штука,814.0,NaN,NaN,None,1


Добавим информацию об изначальных заказчиках в выборку

In [12]:
notice_customers_for_contract = pd.read_sql_query(
    """
    SELECT c.guid as contract_guid, pn.customer_inn as notice_customer_inn,
        pn.placer_inn as notice_placer_inn, pn.detached_org_inn as notice_detached_org_inn
    FROM contract c
    INNER JOIN (
        SELECT contract_reg_number, MAX(version) ver
        FROM contract
        WHERE emergency = 1
        AND lot_guid is not null
        GROUP BY contract_reg_number
    ) tmp ON c.contract_reg_number = tmp.contract_reg_number AND c.version = tmp.ver
    INNER JOIN (
        --если несколько извещений с одинаковым номером, выбераем последнее
        SELECT c2.guid, c2.purchase_notice_info_number, MAX(pn2.create_date_time) as max_date
        FROM contract c2
        INNER JOIN purchase_notice pn2 ON pn2.registration_number = c2.purchase_notice_info_number
        WHERE c2.emergency = 1
        AND c2.purchase_notice_info_number is not null
        GROUP BY c2.guid, c2.purchase_notice_info_number
    ) tmp2 ON c.guid = tmp2.guid
    LEFT JOIN purchase_notice pn ON pn.create_date_time = tmp2.max_date
        AND pn.registration_number = tmp2.purchase_notice_info_number
    WHERE c.emergency = 1
    GROUP BY c.guid, pn.customer_inn, pn.placer_inn, pn.detached_org_inn
    """,
    conn
)

notice_customers_for_contract

,contract_guid,notice_customer_inn,notice_placer_inn,notice_detached_org_inn
0,000efbce-29cc-4fda-ba27-e5b895f5b6ad,6684021776,6684021776,None
1,0010055e-61aa-42c1-9ae5-cfadf002275c,6660000400,6660000400,None
2,001170e0-57c5-4d89-89f2-e607079cd321,3821005886,3821005886,None
3,00161a73-4f64-432b-915a-6d751c91d5a1,8603076205,8603076205,None
4,0019ca0a-9df7-43fe-a57a-1a76ace715c4,3525147496,3525147496,None
...,...,...,...,...
7641,ffd13b8b-ffef-4b0d-8fa5-3214f3e44e2b,2312210286,2312210286,None
7642,ffda639d-aada-44de-974a-4ce9ac345a55,7723011906,7723011906,None
7643,ffe52c52-50b9-4962-8392-28ab996f7d62,6829018032,6829018032,None
7644,ffeed1b5-bd37-4706-8109-95a9367e499e,6731033838,6731033838,None


In [13]:
dataset = pd.merge(
    dataset,
    notice_customers_for_contract,
    how='left',
    on='contract_guid'
)

dataset

,contract_guid,contract_registration_number,notice_not_placed_by_fz223p5s4,contract_extension,contract_prolongation,customer_appeale_or_needs_approval,customer_approval_or_antimonopoly_descision_date,contract_plan_position_guid,contract_create_date_time,contract_customer_inn,...,position_okei_code,position_okei_name,position_qty,position_unit_price,position_rub_unit_price,position_currency_code,contracts_for_lot,notice_customer_inn,notice_placer_inn,notice_detached_org_inn
0,000efbce-29cc-4fda-ba27-e5b895f5b6ad,56684021776200003580001,0,1,0,0,None,None,2020-12-02 05:02:53,6684021776,...,778,Упаковка,700.0,750.0,NaN,None,1,6684021776,6684021776,None
1,000efbce-29cc-4fda-ba27-e5b895f5b6ad,56684021776200003580001,0,1,0,0,None,None,2020-12-02 05:02:53,6684021776,...,778,Упаковка,200.0,1600.0,NaN,None,1,6684021776,6684021776,None
2,000efbce-29cc-4fda-ba27-e5b895f5b6ad,56684021776200003580001,0,1,0,0,None,None,2020-12-02 05:02:53,6684021776,...,778,Упаковка,500.0,940.0,NaN,None,1,6684021776,6684021776,None
3,000efbce-29cc-4fda-ba27-e5b895f5b6ad,56684021776200003580001,0,1,0,0,None,None,2020-12-02 05:02:53,6684021776,...,778,Упаковка,66.0,2595.0,NaN,None,1,6684021776,6684021776,None
4,000efbce-29cc-4fda-ba27-e5b895f5b6ad,56684021776200003580001,0,1,0,0,None,None,2020-12-02 05:02:53,6684021776,...,778,Упаковка,6.0,17050.0,NaN,None,1,6684021776,6684021776,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11305,ffd13b8b-ffef-4b0d-8fa5-3214f3e44e2b,52312210286200000990001,0,0,0,0,None,None,2020-12-08 11:26:50,2312210286,...,None,None,NaN,NaN,NaN,None,1,2312210286,2312210286,None
11306,ffda639d-aada-44de-974a-4ce9ac345a55,67723011906200009850003,0,0,0,0,None,None,2020-11-11 10:04:03,7723011906,...,796,Штука,44.0,NaN,NaN,None,1,7723011906,7723011906,None
11307,ffe52c52-50b9-4962-8392-28ab996f7d62,56829018032200000820001,0,0,0,0,None,None,2020-07-15 06:35:10,6829018032,...,None,None,NaN,NaN,NaN,None,1,6829018032,6829018032,None
11308,ffeed1b5-bd37-4706-8109-95a9367e499e,56731033838200000460001,0,0,0,0,None,None,2020-12-21 07:26:42,6731033838,...,796,Штука,814.0,NaN,NaN,None,1,6731033838,6731033838,None


Добавим информацию о том, сколько позиций в каждом контракте

In [14]:
contract_positions_count = pd.read_sql_query(
    """
    SELECT c.guid as contract_guid, COUNT(*) as contract_positions_count
    FROM contract c
    INNER JOIN (
        SELECT contract_reg_number, MAX(version) ver
        FROM contract
        WHERE emergency = 1
        AND lot_guid is not null
        GROUP BY contract_reg_number
    ) tmp ON c.contract_reg_number = tmp.contract_reg_number AND c.version = tmp.ver
    INNER JOIN purchase_notice_lot pnl ON c.lot_guid = pnl.guid
    LEFT JOIN position_to_contract ptc on c.guid = ptc.contract_guid
    LEFT JOIN contract_position cpos on ptc.position_guid = cpos.guid
    GROUP BY c.guid
    """,
    conn
)

contract_positions_count

,contract_guid,contract_positions_count
0,000efbce-29cc-4fda-ba27-e5b895f5b6ad,11
1,0010055e-61aa-42c1-9ae5-cfadf002275c,1
2,001170e0-57c5-4d89-89f2-e607079cd321,1
3,00161a73-4f64-432b-915a-6d751c91d5a1,1
4,0019ca0a-9df7-43fe-a57a-1a76ace715c4,1
...,...,...
7641,ffd13b8b-ffef-4b0d-8fa5-3214f3e44e2b,1
7642,ffda639d-aada-44de-974a-4ce9ac345a55,1
7643,ffe52c52-50b9-4962-8392-28ab996f7d62,1
7644,ffeed1b5-bd37-4706-8109-95a9367e499e,1


In [15]:
dataset = pd.merge(
    dataset,
    contract_positions_count,
    how='left',
    on='contract_guid'
)

dataset


,contract_guid,contract_registration_number,notice_not_placed_by_fz223p5s4,contract_extension,contract_prolongation,customer_appeale_or_needs_approval,customer_approval_or_antimonopoly_descision_date,contract_plan_position_guid,contract_create_date_time,contract_customer_inn,...,position_okei_name,position_qty,position_unit_price,position_rub_unit_price,position_currency_code,contracts_for_lot,notice_customer_inn,notice_placer_inn,notice_detached_org_inn,contract_positions_count
0,000efbce-29cc-4fda-ba27-e5b895f5b6ad,56684021776200003580001,0,1,0,0,None,None,2020-12-02 05:02:53,6684021776,...,Упаковка,700.0,750.0,NaN,None,1,6684021776,6684021776,None,11
1,000efbce-29cc-4fda-ba27-e5b895f5b6ad,56684021776200003580001,0,1,0,0,None,None,2020-12-02 05:02:53,6684021776,...,Упаковка,200.0,1600.0,NaN,None,1,6684021776,6684021776,None,11
2,000efbce-29cc-4fda-ba27-e5b895f5b6ad,56684021776200003580001,0,1,0,0,None,None,2020-12-02 05:02:53,6684021776,...,Упаковка,500.0,940.0,NaN,None,1,6684021776,6684021776,None,11
3,000efbce-29cc-4fda-ba27-e5b895f5b6ad,56684021776200003580001,0,1,0,0,None,None,2020-12-02 05:02:53,6684021776,...,Упаковка,66.0,2595.0,NaN,None,1,6684021776,6684021776,None,11
4,000efbce-29cc-4fda-ba27-e5b895f5b6ad,56684021776200003580001,0,1,0,0,None,None,2020-12-02 05:02:53,6684021776,...,Упаковка,6.0,17050.0,NaN,None,1,6684021776,6684021776,None,11
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11305,ffd13b8b-ffef-4b0d-8fa5-3214f3e44e2b,52312210286200000990001,0,0,0,0,None,None,2020-12-08 11:26:50,2312210286,...,None,NaN,NaN,NaN,None,1,2312210286,2312210286,None,1
11306,ffda639d-aada-44de-974a-4ce9ac345a55,67723011906200009850003,0,0,0,0,None,None,2020-11-11 10:04:03,7723011906,...,Штука,44.0,NaN,NaN,None,1,7723011906,7723011906,None,1
11307,ffe52c52-50b9-4962-8392-28ab996f7d62,56829018032200000820001,0,0,0,0,None,None,2020-07-15 06:35:10,6829018032,...,None,NaN,NaN,NaN,None,1,6829018032,6829018032,None,1
11308,ffeed1b5-bd37-4706-8109-95a9367e499e,56731033838200000460001,0,0,0,0,None,None,2020-12-21 07:26:42,6731033838,...,Штука,814.0,NaN,NaN,None,1,6731033838,6731033838,None,1


Добавим в выборку информацию о поставщиках


In [16]:
lot_suppliers = pd.read_sql_query(
    """
    SELECT lot_guid, si2.inn as supplier_inn,
     si2.name as supplier_name, si2.ogrn as supplier_ogrn, si2.type as supplier_type
    FROM (
        SELECT lot_guid, MAX(supplier_guid) as supplier_guid
        FROM (
            SELECT c.lot_guid
            FROM contract c
            INNER JOIN (
                SELECT contract_reg_number, MAX(version) ver
                FROM contract
                WHERE emergency = 1
                AND lot_guid is not null
                GROUP BY contract_reg_number
            ) tmp ON c.contract_reg_number = tmp.contract_reg_number AND c.version = tmp.ver
            GROUP BY c.lot_guid
        ) tmp1
        INNER JOIN protocol_lot pl ON tmp1.lot_guid = pl.guid
        LEFT JOIN protocol_lot_applications pla on pl.guid = pla.protocol_lot_guid
        LEFT JOIN application a on pla.id = a.protocol_lot_applications_id
        LEFT JOIN supplier_info si on a.supplier_guid = si.guid
        WHERE (contract_signed = 1 or
         (a.winner_indication = 'Победитель'and contract_signed is null))
         and (supplier_guid is not null)
        GROUP BY lot_guid
    ) sup
    LEFT JOIN supplier_info si2 ON si2.guid = sup.supplier_guid;
    """,
    conn
)

lot_suppliers

,lot_guid,supplier_inn,supplier_name,supplier_ogrn,supplier_type
0,001180aa-ec39-fe24-e63b-de59c6c8b320,7717729216,"ОБЩЕСТВО С ОГРАНИЧЕННОЙ ОТВЕТСТВЕННОСТЬЮ ""ВЫСО...",1127746547895,Юридическое лицо
1,0062ff81-366a-467f-8b5e-8b8075bb08b3,6454074043,"ООО ""ГорЭнергоСервис""",1056405417463,Юридическое лицо
2,00cb1cf0-322f-46da-9de0-2fa76bfe22ae,7723733387,"ООО ""ДЖОДАС ЭКСПОИМ""",1097746637460,Юридическое лицо
3,00ededaa-1b73-4387-8f30-2d56ab2a2459,5005060517,ООО «СТРОЙНАДЗОР»,1155005000281,Юридическое лицо
4,00ff6c74-d199-478a-89ca-9ee62384335d,6454074043,"ООО ""ГорЭнергоСервис""",1056405417463,Юридическое лицо
...,...,...,...,...,...
1166,ff46f520-4f87-4c0b-b114-a7527bf40b6d,5208005007,Общество с ограниченной ответственностью «Энер...,1105252001381,Юридическое лицо
1167,ff6bfde8-e793-4c0c-8888-0433d8fa1ffe,3801069539,"ООО ""Арника-Пром-Сервис""",1033800547978,Юридическое лицо
1168,ffaa3d53-69f7-48e7-a679-8cbc44eb7bbd,7725628067,"ООО ""БионаФарм""",1087746155111,Юридическое лицо
1169,ffb5bde5-3196-49ec-b1d8-9385a4bed2d0,7725628067,"ООО ""БионаФарм""",1087746155111,Юридическое лицо


In [17]:
dataset = pd.merge(
    dataset,
    lot_suppliers,
    how='left',
    on='lot_guid'
)

dataset

,contract_guid,contract_registration_number,notice_not_placed_by_fz223p5s4,contract_extension,contract_prolongation,customer_appeale_or_needs_approval,customer_approval_or_antimonopoly_descision_date,contract_plan_position_guid,contract_create_date_time,contract_customer_inn,...,position_currency_code,contracts_for_lot,notice_customer_inn,notice_placer_inn,notice_detached_org_inn,contract_positions_count,supplier_inn,supplier_name,supplier_ogrn,supplier_type
0,000efbce-29cc-4fda-ba27-e5b895f5b6ad,56684021776200003580001,0,1,0,0,None,None,2020-12-02 05:02:53,6684021776,...,None,1,6684021776,6684021776,None,11,NaN,NaN,NaN,NaN
1,000efbce-29cc-4fda-ba27-e5b895f5b6ad,56684021776200003580001,0,1,0,0,None,None,2020-12-02 05:02:53,6684021776,...,None,1,6684021776,6684021776,None,11,NaN,NaN,NaN,NaN
2,000efbce-29cc-4fda-ba27-e5b895f5b6ad,56684021776200003580001,0,1,0,0,None,None,2020-12-02 05:02:53,6684021776,...,None,1,6684021776,6684021776,None,11,NaN,NaN,NaN,NaN
3,000efbce-29cc-4fda-ba27-e5b895f5b6ad,56684021776200003580001,0,1,0,0,None,None,2020-12-02 05:02:53,6684021776,...,None,1,6684021776,6684021776,None,11,NaN,NaN,NaN,NaN
4,000efbce-29cc-4fda-ba27-e5b895f5b6ad,56684021776200003580001,0,1,0,0,None,None,2020-12-02 05:02:53,6684021776,...,None,1,6684021776,6684021776,None,11,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11305,ffd13b8b-ffef-4b0d-8fa5-3214f3e44e2b,52312210286200000990001,0,0,0,0,None,None,2020-12-08 11:26:50,2312210286,...,None,1,2312210286,2312210286,None,1,2370005720,"ОБЩЕСТВО С ОГРАНИЧЕННОЙ ОТВЕТСТВЕННОСТЬЮ ""ЭНЕР...",1162370050446,Юридическое лицо
11306,ffda639d-aada-44de-974a-4ce9ac345a55,67723011906200009850003,0,0,0,0,None,None,2020-11-11 10:04:03,7723011906,...,None,1,7723011906,7723011906,None,1,NaN,NaN,NaN,NaN
11307,ffe52c52-50b9-4962-8392-28ab996f7d62,56829018032200000820001,0,0,0,0,None,None,2020-07-15 06:35:10,6829018032,...,None,1,6829018032,6829018032,None,1,NaN,NaN,NaN,NaN
11308,ffeed1b5-bd37-4706-8109-95a9367e499e,56731033838200000460001,0,0,0,0,None,None,2020-12-21 07:26:42,6731033838,...,None,1,6731033838,6731033838,None,1,NaN,NaN,NaN,NaN


Выгрузим датасет в эксель

In [18]:
dataset.to_excel("output.xlsx", sheet_name="Main")

Найдём информацию о заявках, на интересующие нас лоты


In [19]:
lot_applications = pd.read_sql_query(
    """
    SELECT tmp1.lot_guid, pl.initial_sum as lot_initial_sum, pl.commodity_price as lot_commodity_price,
       pl.max_contract_price as lot_max_contract_price, pl.non_price as lot_non_price,
       pl.currency_code as lot_currency_code,
       a.application_date as application_date, a.not_dishonest as application_not_dishonest,
       a.provider as application_provider, a.supplier_guid as supplier_guid, si.inn as supplier_inn,
       si.name as supplier_name, si.ogrn as supplier_ogrn, si.type as supplier_type,
       a.price as application_price, a.currency_code as application_currency_code,
       a.accepted as application_accepted, a.application_rejection_reason_code as application_rejection_reason_code,
       a.winner_indication as application_winner_indication, a.contract_signed as application_contract_signed,
       a.additional_price as application_additional_price, a.rating as application_rating,
       a.assessment_result as application_assessment_result,
       a.commission_decision_place as application_commission_decision_place
    FROM (
        SELECT c.lot_guid
        FROM contract c
        INNER JOIN (
            SELECT contract_reg_number, MAX(version) ver
            FROM contract
            WHERE emergency = 1
            AND lot_guid is not null
            GROUP BY contract_reg_number
        ) tmp ON c.contract_reg_number = tmp.contract_reg_number AND c.version = tmp.ver
        GROUP BY c.lot_guid
    ) tmp1
    INNER JOIN protocol_lot pl ON tmp1.lot_guid = pl.guid
    INNER JOIN protocol_lot_applications pla on pl.guid = pla.protocol_lot_guid
    INNER JOIN application a on pla.id = a.protocol_lot_applications_id
    LEFT JOIN supplier_info si on a.supplier_guid = si.guid
    ;
    """,
    conn
)

lot_applications

,lot_guid,lot_initial_sum,lot_commodity_price,lot_max_contract_price,lot_non_price,lot_currency_code,application_date,application_not_dishonest,application_provider,supplier_guid,...,application_price,application_currency_code,application_accepted,application_rejection_reason_code,application_winner_indication,application_contract_signed,application_additional_price,application_rating,application_assessment_result,application_commission_decision_place
0,001180aa-ec39-fe24-e63b-de59c6c8b320,532546.68,None,NaN,NaN,RUB,2021-09-29 11:18:52,1.0,1.0,1c9f992c-e5ef-45c3-9a26-5cae096c0914,...,528360.00,RUB,Допущен,None,None,NaN,NaN,NaN,3.0,3.0
1,001180aa-ec39-fe24-e63b-de59c6c8b320,532546.68,None,NaN,NaN,RUB,2021-09-30 09:40:50,1.0,1.0,5ea6fc6f-832d-4ffb-b16e-dc6dd159a4c6,...,528179.80,RUB,Допущен,None,None,1.0,NaN,NaN,1.0,1.0
2,001180aa-ec39-fe24-e63b-de59c6c8b320,532546.68,None,NaN,NaN,RUB,2021-09-30 06:20:07,1.0,1.0,d9ce1e3e-75d0-45b3-abcc-86d3f5fd5066,...,528286.31,RUB,Допущен,None,None,NaN,NaN,NaN,2.0,2.0
3,001180aa-ec39-fe24-e63b-de59c6c8b320,532546.68,None,NaN,NaN,RUB,2021-09-30 08:31:16,1.0,1.0,f6d6a2c5-0644-4bf6-b351-55d43380fd02,...,532546.68,RUB,Допущен,None,None,NaN,NaN,NaN,4.0,4.0
4,0024ab3a-2d0b-1777-ac85-fe8b9ab39c2d,3700412.11,None,NaN,NaN,RUB,2020-07-28 06:36:41,NaN,NaN,None,...,NaN,None,Не указывается в данном протоколе,None,Не указывается в данном протоколе,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5494,ffb5bde5-3196-49ec-b1d8-9385a4bed2d0,2050145.44,None,NaN,0.0,RUB,2020-11-30 12:30:00,1.0,1.0,48a73f6e-5714-4792-87c3-0ffd0597131d,...,2050145.44,RUB,Допущен,STRUCTURE_CONTENT_DESIGN,None,1.0,2050145.44,1.0,NaN,1.0
5495,ffc9c985-1386-45ab-86f3-9b6590a600aa,2145000.00,None,NaN,NaN,RUB,2021-01-19 09:00:00,NaN,NaN,None,...,2145000.00,RUB,None,None,None,NaN,NaN,NaN,NaN,NaN
5496,ffdd300a-57e0-493f-8f46-460d1ff65276,120000.00,None,NaN,NaN,RUB,2020-10-15 08:00:00,NaN,NaN,None,...,120000.00,RUB,None,None,None,NaN,NaN,NaN,NaN,NaN
5497,ffe29952-0bec-4deb-8422-0a118b0842e7,525000.00,None,NaN,0.0,RUB,2021-09-29 08:00:00,1.0,0.0,dbd4a182-5e48-4976-8016-510706a4f1cf,...,525000.00,RUB,Допущен,STRUCTURE_CONTENT_DESIGN,None,1.0,NaN,NaN,NaN,1.0


Выгрузим заявки в эксель

In [20]:
lot_applications.to_excel("output.xlsx", sheet_name="Applications")

Проанализируем исполнение договоров в нашей выборке:

In [21]:
failed_contracts = covid_contracts_performance_with_lot[
    covid_contracts_performance_with_lot["contract_is_complete"] == 0]
completed_contracts = covid_contracts_performance_with_lot[
    covid_contracts_performance_with_lot["contract_is_complete"] == 1]

contracts_count = covid_contracts_performance_with_lot.shape[0]
failed_contracts_count = failed_contracts.shape[0]
completed_contracts_count = completed_contracts.shape[0]

print(f'Всего договоров: {contracts_count}')
print(f'Всего завершенных договоров: {failed_contracts_count + completed_contracts_count}')
print(f'Всего расторженных договоров: {failed_contracts_count}')
print(f'Всего исполненных договоров: {completed_contracts_count}')
print(
    f'Процент расторженных договоров от завершенных: {failed_contracts_count / (failed_contracts_count + completed_contracts_count) * 100}%')
print(f'Процент расторженных договоров от всех: {failed_contracts_count / contracts_count * 100}%')

Всего договоров: 7646
Всего завершенных договоров: 5522
Всего расторженных договоров: 198
Всего исполненных договоров: 5324
Процент расторженных договоров от завершенных: 3.5856573705179287%
Процент расторженных договоров от всех: 2.5895893277530733%


Проанализируем в каком количестве позиций присутствует цена за единицу

In [22]:
positions_count = covid_contracts_positions_with_lot.shape[0]
unit_price_positions_count = covid_contracts_positions_with_lot[covid_contracts_positions_with_lot["position_unit_price"].notnull()].shape[0]

print(f'Всего позиций: {positions_count}')
print(f'Всего позиций с указанной ценой: {unit_price_positions_count}')
print(f'Процент позиций с указанной ценой: {unit_price_positions_count / positions_count * 100}%')

Всего позиций: 11310
Всего позиций с указанной ценой: 4482
Процент позиций с указанной ценой: 39.62864721485411%
